## Imports

In [31]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
from tabula.io import read_pdf
import re
import os

## Import data from PDF

In [15]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

## Pandas Manipulation

In [16]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [17]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

## Geocoder Setup and Execution

In [18]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [19]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [20]:
df['loc']

1     (788 NE 23rd St #1101, Miami, FL 33137, USA, (...
2     (789 Crandon Blvd APT 504, Key Biscayne, FL 33...
3     (21200 NE 38th Ave APT 802, Aventura, FL 33180...
4     (1445 16th St APT 1101, Miami Beach, FL 33139,...
5     (3000 Island Blvd #2503, Aventura, FL 33160, U...
6     (7000 Island Blvd APT 1906, Aventura, FL 33160...
7     (6103 Aqua Ave APT 502, Miami Beach, FL 33141,...
8     (3131 NE 7th Ave #1503, Miami, FL 33137, USA, ...
9     (5225 Collins Ave APT 1221, Miami Beach, FL 33...
10    (9 Island Ave APT 1905, Miami Beach, FL 33139,...
Name: loc, dtype: object

In [21]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [22]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,The Elysee\r788 NE 23rd St 1101\rMiami,6/1/2022,7/5/2022,2,"$2,970,000.00",3310,"$1,082.76",Felise Eber,Coldwell Banker Realty,Pedro Velasco,Global Luxury Realty LLC,The Elysee 788 NE 23rd St 1101 Miami,The Elysee,788 NE 23rd St 1101 Miami,"(788 NE 23rd St #1101, Miami, FL 33137, USA, (...","(25.7990651, -80.18554160000001, 0.0)",25.799065,-80.185542,0.0
2,Ocean Club\r789 Crandon Blvd 504\rKey Biscayne,2/22/2022,7/6/2022,83,"$2,800,000.00",2500,"$1,120.00",Liz Hogan,"Compass Florida, Inc",Elena Chacon,BHHS EWM Realty,Ocean Club 789 Crandon Blvd 504 Key Biscayne,Ocean Club,789 Crandon Blvd 504 Key Biscayne,"(789 Crandon Blvd APT 504, Key Biscayne, FL 33...","(25.6868145, -80.15900479999999, 0.0)",25.686815,-80.159005,0.0
3,Atlantic 1 at the Point\r21200 NE 38th Ave 802...,3/20/2022,7/5/2022,19,"$1,850,000.00",2733,$676.91,Cassio Galiza PA,Podium Realty Group Inc,Lucas Pucheta,BM2 Realty,Atlantic 1 at the Point 21200 NE 38th Ave 802 ...,Atlantic,1 at the Point 21200 NE 38th Ave 802 Aventura,"(21200 NE 38th Ave APT 802, Aventura, FL 33180...","(25.9730645, -80.1250513, 0.0)",25.973064,-80.125051,0.0
4,Capri South Beach Condo\r1445 16th St 1101\rMi...,2/28/2022,7/5/2022,66,"$1,800,000.00",1128,"$1,595.74",Jeri Jenkins,Coldwell Banker Realty,Pamela Diaz-De-Leon,Brown Harris Stevens,Capri South Beach Condo 1445 16th St 1101 Miam...,Capri South Beach Condo,1445 16th St 1101 Miami Beach,"(1445 16th St APT 1101, Miami Beach, FL 33139,...","(25.7891852, -80.14467069999999, 0.0)",25.789185,-80.144671,0.0
5,Williams Island\r3000 Island Blvd 2503\rAventura,3/28/2022,7/5/2022,23,"$1,525,000.00",2605,$585.41,Nicole Shein,Williams Island Realty LLC,Joel Matus,Williams Island Realty LLC,Williams Island 3000 Island Blvd 2503 Aventura,Williams Island,3000 Island Blvd 2503 Aventura,"(3000 Island Blvd #2503, Aventura, FL 33160, U...","(25.9406051, -80.1344671, 0.0)",25.940605,-80.134467,0.0
6,Williams Island\r7000 Island Blvd 1906\rAventura,5/19/2022,7/7/2022,5,"$1,500,000.00",2390,$627.62,Joel Matus,Williams Island Realty LLC,Nancy Goldfarb,Douglas Elliman,Williams Island 7000 Island Blvd 1906 Aventura,Williams Island,7000 Island Blvd 1906 Aventura,"(7000 Island Blvd APT 1906, Aventura, FL 33160...","(25.9443867, -80.1356782, 0.0)",25.944387,-80.135678,0.0
7,Aqua at Allison Island\r6103 Aqua Ave 502\rMia...,5/19/2022,7/5/2022,6,"$1,285,000.00",1755,$732.19,Luisa Pizzorni,BHHS EWM Realty,Luisa Pizzorni,BHHS EWM Realty,Aqua at Allison Island 6103 Aqua Ave 502 Miami...,Aqua at Allison Island,6103 Aqua Ave 502 Miami Beach,"(6103 Aqua Ave APT 502, Miami Beach, FL 33141,...","(25.8432662, -80.1228728, 0.0)",25.843266,-80.122873,0.0
8,One Paraiso\r3131 NE 7th Ave 1503\rMiami,1/12/2022,7/5/2022,117,"$1,275,000.00",1526,$835.52,Ricardo Wolf,"Wolf Real Estate Group, PLLC",Jonathan Kanarek,One Sotheby's International,One Paraiso 3131 NE 7th Ave 1503 Miami,One Paraiso,3131 NE 7th Ave 1503 Miami,"(3131 NE 7th Ave #1503, Miami, FL 33137, USA, ...","(25.8070469, -80.1855776, 0.0)",25.807047,-80.185578,0.0
9,The Alexander Condo\r5225 Collins Ave 1221\rMi...,11/29/2021,7/5/2022,189,"$1,250,000.00",1800,$694.44,Evelyn Mackenzie,Aria Luxe Inc,Enzo Rosani,Barnes Int'l Realty LLC,The Alexander Condo 5225 Collins Ave 1221 Miam...,The Alexander Condo,5225 Collins Ave 1221 Miami Beach,"(5225 Collins Ave APT 1221, Miami Beach, FL 33...","(25.8298858, -80.12109, 0.0)",25.829886,-80.121090,0.0
10,N0ine Island Ave Condo\r9 Island Ave 1905\rMia...,3/10/2022,7/5/2022,90,"$1,200,000.00",1440,$833.33,Brigitte Lina,One Sotheby's International,Freddie Baigen,Luxe Properties,N0ine Island Ave Condo 9 Island Ave 1905 Miami...,N,0ine Island Ave Condo 9 Island Ave 1905 Miami ...,"(9 Island Ave APT 1905, Miami Beach, FL 33139,...","(25.7900057, -80.1487379, 0.0)",25.790006,-80.14873

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

## Correction Section
To be used if a name is off. Format for making corrections is as follows:

`df.at[index_num,'column_name']=('what you want inserted')`

In [26]:
df.at[10,'building_name']=('Nine Island Avenue')
df.at[10,'address']=('9 Island Ave')

In [27]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('July 3rd - July 9th')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [25]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [29]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [32]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/CondoSalesWeekEnding070922
